In [1]:
import collections
from typing import List, Set


class TicketRule:
    def __init__(self, raw_rule: str):
        self.field, raw_ranges = raw_rule.strip().split(": ")
        self.valid_ranges = []
        for raw_range in raw_ranges.split(" or "):
            self.valid_ranges.append(tuple([int(n) for n in raw_range.split("-")]))
        
    def __repr__(self):
        return f"TicketRule(field={self.field}, valid_ranges={self.valid_ranges})"
    
    def is_valid_value(self, value: int) -> bool:
        """Determine whether value is valid for this ticket rule."""
        any_valid = False
        
        for valid_range in self.valid_ranges:
            if valid_range[0] <= value <= valid_range[1]:
                any_valid = True
        
        return any_valid
    
    def possible_positions(self, valid_tickets: List["Ticket"]) -> Set[int]:
        positions = [True for v in valid_tickets[0].values]
        
        for ticket in valid_tickets:
            for i, value in enumerate(ticket.values):
                if not self.is_valid_value(value):
                    positions[i] = False
                
        return {i for i, p in enumerate(positions) if p}
        
        
    
class Ticket:
    def __init__(self, raw_ticket):
        self.values = [int(v) for v in raw_ticket.strip().split(",")]
    
    def __repr__(self):
        return f"Ticket(values={self.values})"
    
    def error_rate_for_rules(self, rules: List[TicketRule]) -> int:
        error_rate = 0
        
        for value in self.values:
            is_valid = False
                    
            for rule in rules:
                if rule.is_valid_value(value):
                    is_valid = True
                    
            if not is_valid:
                error_rate += value
                
        return error_rate
    
    def is_valid_for_rules(self, rules: List[TicketRule]) -> bool:
        return self.error_rate_for_rules(rules) == 0

In [2]:
filename = "day-16-input.txt"

with open(filename) as file:
    raw_rules, raw_my_ticket, raw_nearby_tickets = file.read().split("\n\n")
    
rules = [TicketRule(raw_rule) for raw_rule in raw_rules.splitlines()]
my_ticket = Ticket(raw_my_ticket.splitlines()[1])
nearby_tickets = [Ticket(raw_ticket) for raw_ticket in raw_nearby_tickets.splitlines()[1:]]

# Part 1

In [3]:
error_rate = 0

for ticket in nearby_tickets:
    error_rate += ticket.error_rate_for_rules(rules)
    
print("Ticket scanning error rate:", error_rate)

Ticket scanning error rate: 24980


# Part 2

In [4]:
valid_tickets = [
    ticket for ticket in nearby_tickets if ticket.is_valid_for_rules(rules)
]

In [5]:
# index -> set of field names
position_to_field = collections.defaultdict(set)
# field name -> possible indices
field_to_positions = {}

for rule in rules:
    positions = rule.possible_positions(valid_tickets)
    for pos in positions:
        position_to_field[pos].add(rule.field)
    field_to_positions[rule.field] = positions

# Narrow down constraints
# Note: the two dicts above are going to get mutated
num_determined_fields = 0
num_fields = len(field_to_positions)

final_positions = ["" for i in range(num_fields)]
while num_determined_fields < num_fields:
    
    # Find field that has clear position
    for field, positions in field_to_positions.items():
        if len(positions) == 1:
            break
    
    # Bookkeep -- mutating the two dicts.
    pos = positions.pop()
    field_to_positions.pop(field)
    position_to_field[pos].remove(field)
    
    final_positions[pos] = field
    num_determined_fields += 1
    
    
    for other_field in position_to_field[pos]:
        field_to_positions[other_field].remove(pos)
        if len(field_to_positions[other_field]) == 0:
            field_to_positions.pop(other_field)
            
    position_to_field.pop(pos)
    
# Now that we've determined the field positions,
# we can analyze our ticket.
product = 1
for value, field in zip(my_ticket.values, final_positions):
    if field.startswith("departure"):
        product *= value
        
print("The answer to part 2 is:", product)

The answer to part 2 is: 809376774329
